## Benchamrking a problems sets from pagmo2



In [6]:
import datetime
import time
import sys
sys.path.append('..')

import pandas as pd

import pygmo as pg
import plotly.express as px
from src.generator import SamplesGenerator

In [144]:
# ----------------------------- Problems param
# DIM = range(2,30)
# ID = range(1,9)
# OBJ = range(2, 30)

ID = 4
DIM = 10
OBJ = 2
SEED = 1

# ----------------------------- Algorithm param

POP_SIZE = 96
GEN = 1000

# ======================================================================




def experiment(problem_name:str,
              prob_id: int, 
              prob_dim: int,
              obj: int, 
              pop_size: int, 
              gen: int, 
              algo_name = 'nsga2', 
              seed=None):
    
    result = {
        "problem_name" : '',
        "seed" : seed,
        "problem_id" : prob_id,
        "objectives" : '',
        "feature_dim" : '',
        "pop_size" : pop_size,
        "generation" : gen,
        "pop_x" : '',
        "pop_f" : '',
        "evolve_time" : '',
        "total_time" : '',
        "date" : '',
        "p_distance" : '',
        "hypervolume": '',
        'error' : '',
        'algo_name': '',
        'final' : False
    }

    # ----------------------                                                            Make problem
    try:
        if problem_name is 'wfg':
            udp = pg.wfg(prob_id=prob_id, dim_dvs=prob_dim, dim_obj=obj, dim_k=obj-1)
        elif problem_name is 'zdt':
            udp = pg.zdt(prob_id=prob_id, param=prob_dim)
        elif problem_name is 'dtlz':
            udp = pg.dtlz(prob_id=prob_id, dim=prob_dim, fdim=obj)
        pro = pg.problem(udp)
    except Exception as err:
        result['error'] = "Init problem: {}".format(err)
        return result


    # is_stochastic alias for has_set_seed()
    t_start = time.time()
    # ----------------------                                                            Initial population
    try:
        pop = pg.population(prob = udp, size = pop_size, seed = seed)
    except Exception as err:
        result['error'] = "Init population: {}".format(err)
        return result

    # ----------------------                                                            Algorithm initialization
    try:
        algo = pg.algorithm(getattr(pg, algo_name)(gen=gen))
    except Exception as err:
        result['error'] = "Init algorithm: {}".format(err)
        return result

    # ----------------------                                                            Solving
    evolve_start = time.time()
    try:
        pop = algo.evolve(pop)   
        score = udp.p_distance(pop) if hasattr(udp, 'p_distance') else None
    except Exception as err:
        result['error'] = "Evolve: {}".format(err)
        return result
    
    # ----------------------                                                            Hypervolume
    try:
        hypervolume = pg.hypervolume(-pop.get_f()).compute([0]*pop.problem.get_nobj())
    except Exception as err:
        result['error'] = "Hypervolume: {}".format(err)
        return result


    # ----------------------                                                            Write results
    try:
        t_end = time.time()
        
        result["problem_name"] = pop.problem.get_name()
        result["objectives"] = pop.problem.get_nobj()
        result["feature_dim"] = pop.problem.get_nx()
        result["pop_x"] = pop.get_x()
        result["pop_f"] = pop.get_f()
        result["evolve_time"] = t_end - evolve_start
        result["total_time"] = t_end - t_start
        result["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        result["final"] = True
        result['algo_name'] = algo_name
        result["p_distance"] = score or None
        result['hypervolume'] = hypervolume or None
        
    except Exception as err:
        result['error'] = "Write results: {}".format(err)
        

    return result

In [98]:
res = experiment('wfg', ID, DIM, OBJ, POP_SIZE, GEN)


In [148]:
from sklearn.model_selection import ParameterGrid
param_grid = {'problem_name': ['wfg', 'dtlz'], 'prob_id': [4, 1], 'prob_dim': [4,15], 'obj': [2], 'pop_size': [24], 'gen': [10], 'algo_name':['moead', 'nsga2', 'nspso']}

res = []
for p in ParameterGrid(param_grid):
    res.append(experiment(**p))
    
pd.DataFrame(res)

,problem_name,seed,problem_id,objectives,feature_dim,pop_size,generation,pop_x,pop_f,evolve_time,total_time,date,p_distance,hypervolume,error,algo_name,final
0,WFG4,None,4,2,4,24,10,"[[1.99999551114794, 2.6591403904691635, 2.6971...","[[2.213920206382855, 0.21395648038152648], [2....",0.002470,0.002694,2020-01-17 00:40,NaN,6.694560e+00,,moead,True
1,DTLZ4,None,4,2,4,24,10,"[[0.6706857914793944, 0.49112503606233493, 0.5...","[[1.0004162502060154, 7.050358533146869e-18], ...",0.001430,0.001530,2020-01-17 00:40,0.006571,7.096707e-01,,moead,True
2,WFG1,None,1,2,4,24,10,"[[0.6031362274454463, 0.7329535834187892, 2.18...","[[2.859850325957927, 0.942766552424506], [2.83...",0.001575,0.001665,2020-01-17 00:40,NaN,6.198284e+00,,moead,True
3,DTLZ1,None,1,2,4,24,10,"[[0.9997407682638016, 0.008815225286480658, 0....","[[150.8977856972966, 0.03912763810035498], [77...",0.001508,0.001571,2020-01-17 00:40,103.640818,1.735151e+03,,moead,True
4,WFG4,None,4,2,15,24,10,"[[1.9998329490967373, 2.760817983923906, 4.434...","[[2.2933656314071325, 0.2948700936895429], [2....",0.002241,0.002408,2020-01-17 00:40,NaN,7.363138e+00,,moead,True
5,DTLZ4,None,4,2,15,24,10,"[[0.2656244790707135, 0.2475447461129051, 0.56...","[[1.170447549572801, 4.912260636043184e-58], [...",0.001225,0.001303,2020-01-17 00:40,0.169111,9.287561e-51,,moead,True
6,WFG1,None,1,2,15,24,10,"[[1.4399003616840984, 1.338236051851729, 2.015...","[[2.9485753226663616, 0.9693366417211114], [2....",0.002359,0.002508,2020-01-17 00:40,NaN,6.582447e+00,,moead,True
7,DTLZ1,None,1,2,15,24,10,"[[0.9999473774360508, 0.11102420294836096, 0.4...","[[577.6588234022904, 0.030399488074325052], [3...",0.001421,0.001496,2020-01-17 00:40,760.003052,6.069979e+04,,moead,True
8,WFG4,None,4,2,4,24,10,"[[0.7461073219862688, 1.6615579343204174, 2.75...","[[0.13189797389880287, 4.094137163140034], [2....",0.001436,0.001500,2020-01-17 00:40,NaN,6.580438e+00,,nsga2,True
9,DTLZ4,None,4,2,4,24,10,"[[0.9919354706028577, 0.6015130195062587, 0.54...","[[0.7768413789082163, 0.6529650697370026], [1....",0.001288,0.001347,2020-01-17 00:40,0.022936,6.001182e-01,,nsga2,True


In [146]:
pd.DataFrame(res).error.values[1]

'Init problem: \nfunction: dtlz\nwhere: /pagmo2/src/problems/dtlz.cpp, 72\nwhat: The problem dimension has to be larger than the number of objectives.\n'